# OpenAI: Azure, APIs, and Agents

## Setup

In [ ]:
# pip install openai-agents
from agents import (
    Agent,
    OpenAIChatCompletionsModel,
    Runner,
    function_tool,
    set_default_openai_api,
    set_default_openai_client,
)
from openai import AzureOpenAI

## Connection

In [ ]:
# params
azure_endpoint = "https://XXXXXXXXXXXXXXXXXXXXXXXXXXX.openai.azure.com/"
api_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
model = "gpt-4o-mini"
# https://learn.microsoft.com/azure/ai-services/openai/reference#rest-api-versioning
api_version = "2024-12-01-preview"
# connection
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    max_retries=3,
    timeout=30,
)

## Chat Completion API (old)

In [ ]:
completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": "You are an assistant that talks like a pirate",
        },
        {
            "role": "user",
            "content": "who was the lead singer of motley crue?",
        },
    ],
    temperature=0.2,
    top_p=0.2,
    max_tokens=200,
)
print(completion.to_dict(), end="\n\n")
print(completion.to_dict()["choices"][0]["message"]["content"])

## Responses API (new)

In [ ]:
response = client.responses.create(
    model=model,
    instructions="You are an assistant that talks like a pirate",
    input="who was the lead singer of Motley Crue?",
    temperature=0.2,
    top_p=0.2,
    max_output_tokens=100,
)
print(response.output_text)

In [ ]:
# chat history and state is implemented via previous_response_id
# chat loop
last_id = None
while True:
    user_input = input("\n> ")
    if user_input.lower() in ["exit", "quit"]:
        break
    if last_id:
        response = client.responses.create(
            model=model,
            input=user_input,
            previous_response_id=last_id,
            max_output_tokens=100,
        )
    else:
        response = client.responses.create(
            model=model,
            input=user_input,
            max_output_tokens=100,
        )
    last_id = response.id
    print("\n" + response.output_text)

In [ ]:
# built-in tools
response_3 = client.responses.create(
    model=model,
    instructions="You are an assistant that talks like a pirate",
    input="who was the lead singer of Motley Crue as of 2025?",
    temperature=0.2,
    top_p=0.2,
    max_output_tokens=200,
    tools=[{"type": "web_search"}],
)
print(response.output_text)

## Agents

In [ ]:
# connection
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    max_retries=1,
    timeout=30,
)
set_default_openai_client(client)

In [ ]:
# functions
@function_tool
def get_weather(city: str):
    if city.lower() == "chicago":
        return f"The weather in {city} is sunny and 98 degrees F."
    elif city.lower() == "boston":
        return f"The weather in {city} in rainy and 75 degrees F."
    else:
        return f"no data on {city}"


@function_tool
def multiply_numbers(number_1: float, number_2: float) -> float:
    return number_1 * number_2

In [ ]:
# agents
weather_agent = Agent(
    name="weather agent",
    instructions="You look up the weather using your get_weather tool",
    tools=[get_weather],
)

math_agent = Agent(
    name="math agent", instructions="You only speak English", tools=[multiply_numbers]
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[weather_agent, math_agent],
)

In [ ]:
# run
result = await Runner.run(
    triage_agent,
    input="what is the temperature in chicago multiplied by the temperature in boston?",
)
print(result.final_output)

## Swarm

In [ ]:
# pip install git+https://github.com/openai/swarm.git
from swarm import Agent, Swarm

In [ ]:
# connection
swarm_client = Swarm(client=client)

In [ ]:
# tools
def transfer_to_agent_b():
    return agent_b

In [ ]:
# agents
agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Haikus.",
)

In [ ]:
# run
response = swarm_client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

print(response.messages[-1]["content"])